*Set up an in-memory SQLite database and create a cursor for executing SQL queries.*

In [2]:
import sqlite3

# create connection (in-memory DB)
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
print("✅ SQLite database created successfully!")


✅ SQLite database created successfully!


*Create a table named EMPLOYEE with empno, ename, job, and salary columns*

In [3]:
cur.execute('''
CREATE TABLE EMPLOYEE(
    empno INTEGER PRIMARY KEY,
    ename TEXT,
    job TEXT,
    salary REAL
)
''')
conn.commit()
print("✅ EMPLOYEE table created!")


✅ EMPLOYEE table created!


*Insert multiple employee records into the EMPLOYEE table*

In [4]:
employees = [
    (101, 'Pavitra', 'Developer', 55000),
    (102, 'Ravi', 'Tester', 48000),
    (103, 'Sneha', 'Manager', 70000),
    (104, 'Kiran', 'Intern', 30000)
]

cur.executemany('INSERT INTO EMPLOYEE VALUES (?, ?, ?, ?)', employees)
conn.commit()
print("✅ Records inserted!")



✅ Records inserted!


*Write an SQL query to fetch all employee records.*

In [5]:
cur.execute('SELECT * FROM EMPLOYEE')
for row in cur.fetchall():
    print(row)


(101, 'Pavitra', 'Developer', 55000.0)
(102, 'Ravi', 'Tester', 48000.0)
(103, 'Sneha', 'Manager', 70000.0)
(104, 'Kiran', 'Intern', 30000.0)


*Select employees whose salary is greater than 50,000.*

In [6]:
cur.execute('SELECT ename, job, salary FROM EMPLOYEE WHERE salary > 50000')
for row in cur.fetchall():
    print(row)


('Pavitra', 'Developer', 55000.0)
('Sneha', 'Manager', 70000.0)


*Update the salary of Ravi to 52000 and delete employee Kiran*

In [7]:
cur.execute("UPDATE EMPLOYEE SET salary = 52000 WHERE ename = 'Ravi'")
cur.execute("DELETE FROM EMPLOYEE WHERE ename = 'Kiran'")
conn.commit()

cur.execute('SELECT * FROM EMPLOYEE')
print("✅ After Update/Delete:")
for row in cur.fetchall():
    print(row)


✅ After Update/Delete:
(101, 'Pavitra', 'Developer', 55000.0)
(102, 'Ravi', 'Tester', 52000.0)
(103, 'Sneha', 'Manager', 70000.0)


*Find the total, average, maximum, and minimum salary.*

In [8]:
cur.execute('SELECT SUM(salary), AVG(salary), MAX(salary), MIN(salary) FROM EMPLOYEE')
total, avg, max_sal, min_sal = cur.fetchone()
print(f"Total={total}, Average={avg}, Max={max_sal}, Min={min_sal}")


Total=177000.0, Average=59000.0, Max=70000.0, Min=52000.0


*Display top 2 highest-paid employees*

In [9]:
cur.execute('SELECT ename, salary FROM EMPLOYEE ORDER BY salary DESC LIMIT 2')
for row in cur.fetchall():
    print(row)


('Sneha', 70000.0)
('Pavitra', 55000.0)


*Create a DEPARTMENT table and perform an INNER JOIN with EMPLOYEE*

In [10]:
cur.execute('''
CREATE TABLE DEPARTMENT(
    deptno INTEGER PRIMARY KEY,
    deptname TEXT
)
''')

departments = [(1, 'IT'), (2, 'HR'), (3, 'Finance')]
cur.executemany('INSERT INTO DEPARTMENT VALUES (?, ?)', departments)
conn.commit()

# Add deptno column temporarily
cur.execute('ALTER TABLE EMPLOYEE ADD COLUMN deptno INTEGER')
cur.execute("UPDATE EMPLOYEE SET deptno = 1 WHERE job='Developer'")
cur.execute("UPDATE EMPLOYEE SET deptno = 2 WHERE job='Tester'")
cur.execute("UPDATE EMPLOYEE SET deptno = 3 WHERE job='Manager'")
conn.commit()

# INNER JOIN query
cur.execute('''
SELECT e.ename, e.job, d.deptname, e.salary
FROM EMPLOYEE e
JOIN DEPARTMENT d ON e.deptno = d.deptno
''')
for row in cur.fetchall():
    print(row)


('Pavitra', 'Developer', 'IT', 55000.0)
('Ravi', 'Tester', 'HR', 52000.0)
('Sneha', 'Manager', 'Finance', 70000.0)


**GROUP BY and Aggregate Functions**

*Show total and average salary for each department*

In [11]:
cur.execute('''
SELECT deptno, COUNT(*) AS total_employees,
       SUM(salary) AS total_salary,
       AVG(salary) AS avg_salary
FROM Employee
GROUP BY deptno
''')

for row in cur.fetchall():
    print(row)


(1, 1, 55000.0, 55000.0)
(2, 1, 52000.0, 52000.0)
(3, 1, 70000.0, 70000.0)


*Find departments whose average salary is above 60,000.*

In [12]:
cur.execute('''
SELECT deptno, AVG(salary) AS avg_salary
FROM Employee
GROUP BY deptno
HAVING avg_salary > 60000
''')
for row in cur.fetchall():
    print(row)


(3, 70000.0)


In [15]:
cur.execute('''
SELECT d.deptname, e.ename, e.job
FROM Department d
LEFT JOIN Employee e ON d.deptno = e.deptno
ORDER BY d.deptname
''')
for row in cur.fetchall():
    print(row)


('Finance', 'Sneha', 'Manager')
('HR', 'Ravi', 'Tester')
('IT', 'Pavitra', 'Developer')


*Display employees sorted by salary (descending).*

In [16]:
cur.execute('SELECT ename, job, salary FROM Employee ORDER BY salary DESC')
for row in cur.fetchall():
    print(row)



('Sneha', 'Manager', 70000.0)
('Pavitra', 'Developer', 55000.0)
('Ravi', 'Tester', 52000.0)


*Show department-wise average salary*

In [17]:
cur.execute('SELECT deptno, AVG(salary) AS avg_salary FROM Employee GROUP BY deptno')
for row in cur.fetchall():
    print(row)


(1, 55000.0)
(2, 52000.0)
(3, 70000.0)


*Departments with average salary > 55,000*

In [18]:
cur.execute('''
SELECT deptno, AVG(salary) AS avg_salary
FROM Employee
GROUP BY deptno
HAVING avg_salary > 55000
''')
for row in cur.fetchall():
    print(row)


(3, 70000.0)


*Display employee names with their department names.*

In [19]:
cur.execute('''
SELECT e.ename, e.job, d.deptname, e.salary
FROM Employee e
JOIN Department d ON e.deptno = d.deptno
''')
for row in cur.fetchall():
    print(row)


('Pavitra', 'Developer', 'IT', 55000.0)
('Ravi', 'Tester', 'HR', 52000.0)
('Sneha', 'Manager', 'Finance', 70000.0)


*Show all departments and their employees (if any)*

In [20]:
cur.execute('''
SELECT d.deptname, e.ename, e.job
FROM Department d
LEFT JOIN Employee e ON d.deptno = e.deptno
ORDER BY d.deptname
''')
for row in cur.fetchall():
    print(row)


('Finance', 'Sneha', 'Manager')
('HR', 'Ravi', 'Tester')
('IT', 'Pavitra', 'Developer')


*Find employees earning above the average salary.*

In [21]:
cur.execute('''
SELECT ename, salary
FROM Employee
WHERE salary > (SELECT AVG(salary) FROM Employee)
''')
for row in cur.fetchall():
    print(row)


('Sneha', 70000.0)


*List employees in the same department as "Pavitra".*

In [22]:
cur.execute('''
SELECT ename, job
FROM Employee
WHERE deptno IN (
    SELECT deptno FROM Employee WHERE ename = 'Pavitra'
)
''')
for row in cur.fetchall():
    print(row)


('Pavitra', 'Developer')


*Find total, average, max, and min salary.*

In [23]:
cur.execute('SELECT SUM(salary), AVG(salary), MAX(salary), MIN(salary) FROM Employee')
print(cur.fetchone())


(177000.0, 59000.0, 70000.0, 52000.0)


*Show all unique job roles*

In [24]:
cur.execute('SELECT DISTINCT job FROM Employee')
for row in cur.fetchall():
    print(row)


('Developer',)
('Tester',)
('Manager',)


*Categorize employees by salary level.*

In [25]:
cur.execute('''
SELECT ename, salary,
CASE
    WHEN salary >= 70000 THEN 'High'
    WHEN salary BETWEEN 50000 AND 69999 THEN 'Medium'
    ELSE 'Low'
END AS Salary_Level
FROM Employee
''')
for row in cur.fetchall():
    print(row)


('Pavitra', 55000.0, 'Medium')
('Ravi', 52000.0, 'Medium')
('Sneha', 70000.0, 'High')


*Get top 3 highest paid employees*

In [26]:
cur.execute('SELECT ename, job, salary FROM Employee ORDER BY salary DESC LIMIT 3')
for row in cur.fetchall():
    print(row)


('Sneha', 'Manager', 70000.0)
('Pavitra', 'Developer', 55000.0)
('Ravi', 'Tester', 52000.0)


*Create a view of employees earning above ₹60,000.*

In [27]:
cur.execute('CREATE VIEW HighEarners AS SELECT ename, job, salary FROM Employee WHERE salary > 60000')
conn.commit()

cur.execute('SELECT * FROM HighEarners')
for row in cur.fetchall():
    print(row)


('Sneha', 'Manager', 70000.0)


*Add a new column "experience" to Employee table.*

In [28]:
cur.execute('ALTER TABLE Employee ADD COLUMN experience INTEGER')
conn.commit()


*Delete employees with salary below ₹45,000.*

In [29]:
cur.execute('DELETE FROM Employee WHERE salary < 45000')
conn.commit()


*Delete the created view.*

In [30]:
cur.execute('DROP VIEW IF EXISTS HighEarners')
conn.commit()

